# Create a custom Studio app image
This notebook implement a process of creating a SageMaker Studio custom app image. The advantage of creating an image and make it available to all SageMaker Studio users is that it creates a consistent environment to use in Studio notebooks. You can also run app images locally which improves reproducability of your workloads.

For details, hands-on examples, and documentation refer to the following resources:
- [SageMaker Studio developer guide](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-byoi.html)
- [Custom image samples GitHub](https://github.com/aws-samples/sagemaker-studio-custom-image-samples/)
- [Use Studio Container Build CLI](https://github.com/aws/amazon-sagemaker-examples/tree/main/aws_sagemaker_studio/sagemaker_studio_image_build)
- [Using the Amazon SageMaker Studio Image Build CLI to build container images from your Studio notebooks](https://aws.amazon.com/blogs/machine-learning/using-the-amazon-sagemaker-studio-image-build-cli-to-build-container-images-from-your-studio-notebooks/)

<div class="alert alert-block alert-warning"> Run this notebook as a SageMaker notebook instance, not as a Studio notebook, because you cannot run Docker commands in a SageMaker Studio notebook or terminal. Studio environment already runs in a Docker container and you cannot run container in container. 
</div>

## Configure environment

In [139]:
import os
import json
import boto3
import sagemaker

Put all container files into a separate directory.

In [140]:
!mkdir container

mkdir: cannot create directory ‘container’: File exists


Get environment metadata:

In [141]:
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        md = json.loads(f.read())
        print(f"Notebook metadata: {md}")

Notebook metadata: {'ResourceArn': 'arn:aws:sagemaker:us-east-1:906545278380:notebook-instance/d2l-notebooks', 'ResourceName': 'd2l-notebooks'}


In [142]:
role = sagemaker.get_execution_role()

In [143]:
repo_name = "conda-env-kernel"
image_name = "conda-env-kernel"
account_id = boto3.client("sts").get_caller_identity()["Account"]
region = md["ResourceArn"].split(":")[3]
full_name = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{repo_name}:{image_name}"

print(full_name)

906545278380.dkr.ecr.us-east-1.amazonaws.com/conda-env-kernel:conda-env-kernel


Create a conda environment specification file. The Conda environment must have a Jupyter kernel package installed, for example, `ipykernel` for Python kernel.

In [164]:
%%writefile container/environment.yml
name: customenv
channels:
  - conda-forge
dependencies:
  - python=3.8.*
  - ipykernel 
  - pip
  - pip:
    - awscli
    - boto3
    - sagemaker
    - scikit-learn==1.0.2
    - scipy==1.8.*
    - seaborn==0.11.2
    - prophet==1.*
    - plotly==5.9.*
    - prophet==1.*
    - metno-locationforecast == 1.*
    - meteostat == 1.*
    - pandas==1.4.*
    - numpy
    - matplotlib==3.5.*
    - geopandas==0.11.*
    - shapely==1.8.*

Overwriting container/environment.yml


Create the second environment specification file:

In [165]:
%%writefile container/environment-env2.yml
name: env2
channels:
  - conda-forge
dependencies:
  - python=3.8.*
  - numpy
  - awscli
  - boto3
  - ipykernel

Writing container/environment-env2.yml


Now create the Dockerfile.

In [169]:
%%writefile container/Dockerfile
FROM continuumio/miniconda3:4.10.3

COPY environment*.yml ./

RUN conda env create -f environment.yml
RUN conda env create -f environment-env2.yml

Overwriting container/Dockerfile


## Build the Docker image

In [167]:
!docker system prune -af

Deleted Containers:
364099ced8e5312cc3f770424ee23adaed74d5e3bc79ecc9d91629b0ead3d5d1

Deleted Images:
untagged: continuumio/miniconda3:4.10.3
untagged: continuumio/miniconda3@sha256:a137c7da98c8680467490e15ac3c54e25db77495be1737076b053a6790ad6082
untagged: 906545278380.dkr.ecr.us-east-1.amazonaws.com/conda-env-kernel:conda-env-kernel
untagged: 906545278380.dkr.ecr.us-east-1.amazonaws.com/conda-env-kernel@sha256:243a83e68a92d759c076ad44e74c4160ad308797f90c07be60d18ee09b78850e
untagged: conda-env-kernel:latest
deleted: sha256:e75acd6b6e6728592abc629b98ec62c19646b34e72730e57519f6a99151db8d5
deleted: sha256:31054d91b38f14f3804ce9557d7af80a3ca26e498dc57c256acdae80c47c547d
deleted: sha256:5ec60ab893af190ecb5f526f6652824241f1601850020889c179ac62faed86a9
deleted: sha256:7510c895034ffd0735e2467f3f395a715ca8fc6dec16a8ef251732fb9c941f6f
deleted: sha256:83812c01cb604e458a933b7c0dd0b6b4bf153aa42a1460bb785b5239e6fffc7c
deleted: sha256:50db6ae4d0d7f5e746bc16dcff1049f6c7dee837df60e98eb8bb27c64c15719f


The following script runs for about 7 minutes.

In [ ]:
%%sh

cd container

# Region, defaults to us-east-1
REGION=$(aws configure get region)
REGION=${REGION:-us-east-1}

ACCOUNT_ID=$(aws sts get-caller-identity --query Account --output text)
REPO_NAME=conda-env-kernel
IMAGE_NAME=conda-env-kernel
FULL_NAME="${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com/${REPO_NAME}:${IMAGE_NAME}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${REPO_NAME}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${REPO_NAME}" > /dev/null
fi

# Login to ECR
aws --region ${REGION} ecr get-login-password | docker login --username AWS --password-stdin ${FULL_NAME}

# Build and push the image
docker build . -t ${IMAGE_NAME} -t ${FULL_NAME}

docker push ${FULL_NAME}

List images in the ECR:

In [148]:
!aws ecr list-images --repository-name {repo_name}

{
    "imageIds": [
        {
            "imageDigest": "sha256:3cf396f406e91c7c0045158faced7e2c689becf543b51a65d76a24b1bdca3b75"
        },
        {
            "imageDigest": "sha256:a92c66c029d27cfdf315e58057d981e7cad811cf424c4ad2783ed06fa8bae741"
        },
        {
            "imageDigest": "sha256:463b0ad96fb187a640a95d45911458b82522f7be4944b18e7357b91a98c8eee1",
            "imageTag": "latest"
        },
        {
            "imageDigest": "sha256:243a83e68a92d759c076ad44e74c4160ad308797f90c07be60d18ee09b78850e",
            "imageTag": "conda-env-kernel"
        },
        {
            "imageDigest": "sha256:73eaed71f6982192f4c450b36fc2e048d0faf5606e67b81c5b40a370962b44aa"
        }
    ]
}


## Use the custom app image with SageMaker Studio
Follow the these steps to use your custom image with Studio:
- Create a SageMaker Image (SMI) with the image in ECR
- Create an App Image Version 
- Create an AppImageConfig 
- Update domain with the AppImageConfig

❗ Everytime you update the image in ECR, a new image version should be created. See [Update image with SageMaker Studio](#update-image-with-sagemaker-studio).

This notebook uses `boto3` SDK, but you can also use `aws cli` to run all commands.

In [170]:
sm = boto3.client("sagemaker")

Create an image configuration file `app-image-config-input.json`. SageMaker Studio will automatically recognize the Conda environments as corresponding kernels named `conda-env-customenv-py` and `conda-env-env2-py`. You can create multiple Conda environments and Studio automatically populates the kernel selection dropdown on the **Set up notebook environment** dialog.

Note the naming convention for `KernelSpecs.Name`. You must specify the kernel name based on the name of your Conda environment: `conda-env-<YOUR CUSTOM ENVIRONMENT NAME FROM environment.yml>-py`.

In [171]:
%%writefile app-image-config-input.json
{
    "AppImageConfigName": "conda-env-kernel-config",
    "KernelGatewayImageConfig": {
        "KernelSpecs": [
            {
                "Name": "conda-env-customenv-py",
                "DisplayName": "Python [conda env: customenv]"
            },
            {
                "Name": "conda-env-env2-py",
                "DisplayName": "Python [conda env: env2]"
            }
        ],
        "FileSystemConfig": {
            "MountPath": "/root",
            "DefaultUid": 0,
            "DefaultGid": 0
        }
    }
}

Overwriting app-image-config-input.json


In [172]:
# load app image config as a variable
with open("app-image-config-input.json", "rb") as f:
    app_image_config = json.loads(f.read())

In [173]:
# create Studio Image
r = sm.create_image(
    Description="custom conda environment",
    DisplayName="Python [conda env: custom_env]",
    ImageName=image_name,
    RoleArn=role,
)

r

{'ImageArn': 'arn:aws:sagemaker:us-east-1:906545278380:image/conda-env-kernel',
 'ResponseMetadata': {'RequestId': '0367765a-ea94-4b66-9b05-0787436ba224',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0367765a-ea94-4b66-9b05-0787436ba224',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '78',
   'date': 'Thu, 27 Oct 2022 11:27:10 GMT'},
  'RetryAttempts': 0}}

In [174]:
# create image version
r = sm.create_image_version(
    BaseImage=full_name,
    ImageName=image_name,
)

r

{'ImageVersionArn': 'arn:aws:sagemaker:us-east-1:906545278380:image-version/conda-env-kernel/1',
 'ResponseMetadata': {'RequestId': '2947e55d-46d6-4826-ad31-fc0f4c21a6b2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2947e55d-46d6-4826-ad31-fc0f4c21a6b2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '95',
   'date': 'Thu, 27 Oct 2022 11:27:13 GMT'},
  'RetryAttempts': 0}}

Check the status of the image version. The status must be `CREATED`. Do not proceed with any other status.

In [175]:
r = sm.describe_image_version(ImageName=image_name)
print(f"Image version status: {r['ImageVersionStatus']}")
assert(r['ImageVersionStatus'] == 'CREATED')

Image version status: CREATED


In [176]:
# create image config
r = sm.create_app_image_config(
    AppImageConfigName=app_image_config["AppImageConfigName"],
    KernelGatewayImageConfig=app_image_config["KernelGatewayImageConfig"],
)

r

{'AppImageConfigArn': 'arn:aws:sagemaker:us-east-1:906545278380:app-image-config/conda-env-kernel-config',
 'ResponseMetadata': {'RequestId': '776a53f5-0212-4a1c-bef1-635c4997846d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '776a53f5-0212-4a1c-bef1-635c4997846d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '105',
   'date': 'Thu, 27 Oct 2022 11:27:17 GMT'},
  'RetryAttempts': 0}}

Update an existing SageMaker domain to use this app image. If you don't have a domain, you must [create a new one](https://docs.aws.amazon.com/sagemaker/latest/dg/onboard-iam.html).

In [177]:
%%writefile update-domain-input.json
{
    "DefaultUserSettings": {
        "KernelGatewayAppSettings": {
            "CustomImages": [
                {
                    "ImageName": "conda-env-kernel",
                    "AppImageConfigName": "conda-env-kernel-config"
                }
            ]
        }
    }
}

Overwriting update-domain-input.json


In [178]:
with open("update-domain-input.json", "rb") as f:
    update_domain_input = json.loads(f.read())

In [179]:
# Get domain_id for the active domain
r = [d for d in sm.list_domains()["Domains"] if d["Status"] == 'InService']
if not len(r):
    raise Exception("You don't have any active domain, create a new one!")
    
if len(r) > 1:
    raise Exception("You have more than one active domain, please manually select one for update!")
    
domain_id = r[0]["DomainId"]

print(domain_id)

d-r8pbvl3oamh6


In [180]:
r = sm.update_domain(
    DomainId=domain_id,
    DefaultUserSettings=update_domain_input["DefaultUserSettings"],
)

r

{'DomainArn': 'arn:aws:sagemaker:us-east-1:906545278380:domain/d-r8pbvl3oamh6',
 'ResponseMetadata': {'RequestId': '10a10cff-1f31-4a50-8684-8d26fbda4d16',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '10a10cff-1f31-4a50-8684-8d26fbda4d16',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '78',
   'date': 'Thu, 27 Oct 2022 11:27:43 GMT'},
  'RetryAttempts': 0}}

There is an about 5 min delay from when a custom image is attached to the domain until the app is visible in the dropdown in **Set up notebook environment**.

## Start a notebook witht the custom app

Start SageMaker Studio, open a new notebook and select your new custom app `conda-env-kernel`:
![](../img/select-kernel.png)

If your app image contains more than one Conda environments, Studio shows these enviroments as selectable kernels in the **Kernel** dropdown:

![](../img/select-kernel-2.png)

After you selected a kernel, Studio automatically activates the corresponding Conda environment in the notebook. You can check in which environment is active with `%conda env list` command in the notebook:

Kernel `customenv`:

![](../img/conda-list-1.png)

Kernel `env2`:

![](../img/conda-list-2.png)

To see the kernels that are installed on the notebook, run this command in the Studio notebook:

In [91]:
!jupyter-kernelspec list

[ListKernelSpecs] WARNING | Config option `kernel_spec_manager_class` not recognized by `ListKernelSpecs`.
Available kernels:
  ir         /home/ec2-user/.local/share/jupyter/kernels/ir
  python3    /home/ec2-user/anaconda3/envs/pytorch_p38/share/jupyter/kernels/python3


## Update image with SageMaker Studio
To update a custom app image in SageMaker domain, for example with a new version of image, follow these steps:
- Build and push a new version of the image to ECR
- Create a new App Image Version
- Re-create App in SageMaker Studio

In [67]:
full_name

'906545278380.dkr.ecr.us-east-1.amazonaws.com/conda-env-kernel:conda-env-kernel'

In [68]:
r = sm.create_image_version(
    BaseImage=full_name,
    ImageName=image_name,
)

r

{'ImageVersionArn': 'arn:aws:sagemaker:us-east-1:906545278380:image-version/conda-env-kernel/3',
 'ResponseMetadata': {'RequestId': 'b8ff92ca-98bc-4797-8b2b-18c7aa2feac2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b8ff92ca-98bc-4797-8b2b-18c7aa2feac2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '95',
   'date': 'Wed, 26 Oct 2022 16:31:01 GMT'},
  'RetryAttempts': 0}}

In [69]:
r = sm.describe_image_version(ImageName=image_name)
print(f"Image version status: {r['ImageVersionStatus']}")
assert(r['ImageVersionStatus'] == 'CREATED')

Image version status: CREATED


Delete your custom app in Studio and restart all associated kernels. You can delete the app either in Studio UX:

![](../img/delete-app-studio.png)

or in AWS console:

![](../img/delete-app-console.png)

After you deleted the custom app, re-open a notebook and select the custom image in the **Set up notebook environment** dialog.

## Local testing
Before attach an app image to a SageMaker domain, you can debug and test your app locally.

Run the following commands in the terminal.

In [ ]:
IMAGE_NAME="conda-env-kernel"
docker run -it "$IMAGE_NAME" bash

For Conda-based images activate an environment:

In [ ]:
conda activate customenv

In the running container list the available kernels:

In [ ]:
jupyter-kernelspec list

If no kernel or `jupyter-kernelspec` is not available, install `ipykernel`:

In [ ]:
pip install ipykernel
python -m ipykernel install --sys-prefix

You must also add `pip install ipykernel` to the Docker file if you don't use Conda environments.

In [ ]:
RUN pip install ipykernel && \
        python -m ipykernel install --sys-prefix

Refer to this [sample notebook](https://github.com/aws-samples/sagemaker-studio-custom-image-samples/blob/main/DEVELOPMENT.md) for more details.

## Clean up
To avoid charges you must stop all active SageMaker notebook instances. Follow the [clean up instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html) in the Developer Guide.

## Resources
- [Available Amazon SageMaker Images](https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-available-images.html)
- [Developer guide for BYOI](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-byoi.html)
- [Custom SageMaker image specifications](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-byoi-specs.html)
- [Conda environments as kernels](https://github.com/aws-samples/sagemaker-studio-custom-image-samples/tree/main/examples/conda-env-kernel-image)
- [Custom app image samples GitHub](https://github.com/aws-samples/sagemaker-studio-custom-image-samples/)
- [How to use Studio Container Build CLI](https://github.com/aws/amazon-sagemaker-examples/tree/main/aws_sagemaker_studio/sagemaker_studio_image_build)
- [Using the Amazon SageMaker Studio Image Build CLI to build container images from your Studio notebooks](https://aws.amazon.com/blogs/machine-learning/using-the-amazon-sagemaker-studio-image-build-cli-to-build-container-images-from-your-studio-notebooks/)
- [Create a new Conda environment in Sagemaker home directory (Amazon EFS)](https://github.com/durgasury/efs_backed_conda)
- [Automating the Setup of SageMaker Studio Custom Images](https://towardsdatascience.com/automating-the-setup-of-sagemaker-studio-custom-images-4a3433fd7148)
- [Activating a Conda environment in your Dockerfile](https://pythonspeed.com/articles/activate-conda-dockerfile/)